In [1]:
from sklearn.decomposition import PCA
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
from computage.configs.datasets_bench_config import *

import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [274]:
# fig, axes = plt.subplots(3, 3, figsize=(16, 16))
# for ax, (gse, conf) in tqdm(zip(axes.flatten(), pca_plot_config.items())):
#     path, cond = conf.values()

#     #prepare data
#     dnam, meta = pd.read_pickle(path, compression='gzip').values()
#     pca = PCA(2)
#     expvar = pca.explained_variance_ratio_
#     Xp = pca.fit_transform(dnam.dropna(axis=1))
#     pca_data = pd.DataFrame({'PC1':Xp[:, 0], 'PC2':Xp[:, 1], 
#                              'Condition':meta['Condition'], 
#                              'Age':meta['Age']})
    

#     #plotting
#     sns.scatterplot(data=pca_data, x='PC1', y='PC2', style='Condition', ax=ax)
#     ax.set_title(gse)


    # ax0.set_title(pca_title)
    # z1_plot = ax0.scatter(Xpa[:, 0], Xpa[:, 1], c=ya, s=50, label=legend_label_train, 
    #                     edgecolors='#333333', cmap=cbar_color_train, zorder=4)
    # z2_plot = ax0.scatter(Xpr[:, 0], Xpr[:, 1], c=yr, marker='s', s=50, label=legend_label_test, 
    #                     edgecolors='#333333', cmap=cbar_color_test, zorder=4)
    # ax0.set_xlabel(f'PCA1: {int(pvar[0]*100)}%')
    # ax0.set_ylabel(f'PCA2: {int(pvar[1]*100)}%')
    # ax0.legend(loc=pca_legend_pos)
    # leg = ax0.get_legend()
    # leg.legendHandles[0].set(edgecolor='#333333')
    # leg.legendHandles[1].set(edgecolor='#333333')
    # ax0.set_xlim(pca_xlim)
    # ax0.set_ylim(pca_ylim)

# full dataset

In [2]:
dnam, meta = pd.read_pickle('/tank/projects/computage/benchmarking/ultra_bench_v1.pkl').values()

In [3]:
#filtering dataset
site_sum = dnam.isna().sum(axis=0)
s = site_sum / dnam.shape[0] < 0.2

cond_list = ['HC', 'AS', 'IBD', 'Rheumatoid arthritis', 'CVA', 'HT', 'HIV', 'T2D', 'OBS', 'AD', 'DS']
filt_indices = meta[(meta['Tissue']=='Blood') & (meta['Condition'].isin(cond_list))].index
data_selected = dnam.loc[filt_indices, s[s].index]
meta_selected = meta.loc[filt_indices]

In [5]:
# bench_dataset = {}
# bench_dataset['data'] = data_selected
# bench_dataset['meta'] = meta_selected
# # pd.to_pickle(train_dataset, f'/tank/projects/computage/train/ultra_train_v1.pkl.gz', compression='gzip')
# pd.to_pickle(bench_dataset, f'/tank/projects/computage/benchmarking/ultra_bench_v1_cut400k.pkl')    

In [30]:
mean_vector = data_selected.mean(axis=0)

In [ ]:
nancols = data_selected.isna().any(axis=0)
for col in tqdm(nancols[nancols].index):
    data_selected.loc[:, col].fillna(mean_vector[col], inplace=True)

In [84]:
data_filled = data_selected.dropna(axis=1)
data_filled.shape

(4967, 402035)

In [85]:
bench_dataset = {}
bench_dataset['data'] = data_filled
bench_dataset['meta'] = meta_selected
pd.to_pickle(bench_dataset, f'/tank/projects/computage/benchmarking/ultra_bench_v1_cut400k_filled.pkl')    

In [86]:
#prepare data
pca = PCA(2)
Xp = pca.fit_transform(data_filled)
expvar = pca.explained_variance_ratio_

In [89]:
pca_data = pd.DataFrame({'PC1':Xp[:, 0], 'PC2':Xp[:, 1], 
                    'Condition':meta_selected['Condition'], 
                    'Age':meta_selected['Age'],
                    'Gender':meta_selected['Gender']
                    })

if 'HC' in meta_selected['Condition'].unique():
    n_HC = meta_selected['Condition'].value_counts()['HC']
else:
    n_HC = 0
n_nHC = meta_selected['Condition'].shape[0] - n_HC
n_cpg = data_filled.shape[1]

#plot data
sub = alt.Chart(pca_data).mark_point(clip=True,).encode(
        alt.X('PC1', axis=alt.Axis(grid=False, tickSize=4, labels=False))\
            .title(f'PC1: {round(expvar[0]* 100, 1) }%'),
        alt.Y('PC2', axis=alt.Axis(grid=False, tickSize=4, labels=False))\
            .title(f'PC2: {round(expvar[1]* 100, 1) }%'),
        # color=alt.Color('Age').scale(scheme="magma"),
        color=alt.Shape('Condition:N'),
        size=alt.Size('Gender:O').scale(domain=['M', 'F'], rangeMin=30, rangeMax=180)
    ).properties(
        title=[f'All data: Blood Tissue, N CpGs = {n_cpg}', f'N healthy = {n_HC}, N non-healthy = {n_nHC}'],
        width=730,
        height=730
    )

sub.save(f'figures/supplement/fullbench_PCA_2.svg')
sub

alt.Chart(...)

In [90]:
pca_data = pd.DataFrame({'PC1':Xp[:, 0], 'PC2':Xp[:, 1], 
                    'Condition':meta_selected['Condition'], 
                    'Age':meta_selected['Age'],
                    'Gender':meta_selected['Gender']
                    })

if 'HC' in meta_selected['Condition'].unique():
    n_HC = meta_selected['Condition'].value_counts()['HC']
else:
    n_HC = 0
n_nHC = meta_selected['Condition'].shape[0] - n_HC
n_cpg = data_filled.shape[1]

#plot data
sub = alt.Chart(pca_data).mark_point(clip=True,).encode(
        alt.X('PC1', axis=alt.Axis(grid=False, tickSize=4, labels=False))\
            .title(f'PC1: {round(expvar[0]* 100, 1) }%'),
        alt.Y('PC2', axis=alt.Axis(grid=False, tickSize=4, labels=False))\
            .title(f'PC2: {round(expvar[1]* 100, 1) }%'),
        color=alt.Color('Age').scale(scheme="magma"),
        shape=alt.Shape('Condition:N'),
        size=alt.Size('Gender:O').scale(domain=['M', 'F'], rangeMin=30, rangeMax=180)
    ).properties(
        title=[f'All data: Blood Tissue, N CpGs = {n_cpg}', f'N healthy = {n_HC}, N non-healthy = {n_nHC}'],
        width=730,
        height=730
    )

sub.save(f'figures/supplement/fullbench_PCA_1.svg')
sub

alt.Chart(...)

# separate datasets

In [272]:
import altair as alt
from glob import glob

ready_plots = glob('figures/supplement/*.svg')
for gse, conf in tqdm(list(datasets_config.items())):
# for gse, conf in tqdm(tmp_config):
    path, cond, _ = conf.values()
    fname = f'figures/supplement/{gse}_PCA_desc.svg'
    if fname in ready_plots:
        continue

    dnam, meta = pd.read_pickle(path, compression='gzip').values()
    dnam = dnam.dropna(axis=1)

    if 'HC' in meta['Condition'].unique():
        n_HC = meta['Condition'].value_counts()['HC']
    else:
        n_HC = 0
    n_nHC = meta['Condition'].shape[0] - n_HC
    n_cpg = dnam.shape[1]

    #prepare data
    pca = PCA(2)
    Xp = pca.fit_transform(dnam)
    expvar = pca.explained_variance_ratio_

    
    if 'Gender' in meta.columns:
        pca_data = pd.DataFrame({'PC1':Xp[:, 0], 'PC2':Xp[:, 1], 
                            'Condition':meta['Condition'], 
                            'Age':meta['Age'],
                            'Gender':meta['Gender']
                            })
        #plot data
        sub = alt.Chart(pca_data).mark_point(clip=True).encode(
                alt.X('PC1', axis=alt.Axis(grid=False, tickSize=4, labels=False))\
                    .title(f'PC1: {round(expvar[0]* 100, 1) }%'),
                alt.Y('PC2', axis=alt.Axis(grid=False, tickSize=4, labels=False))\
                    .title(f'PC2: {round(expvar[1]* 100, 1) }%'),
                color=alt.Color('Age').scale(scheme="reds"),
                shape=alt.Shape('Condition:N'),
                size=alt.Size('Gender:O').scale(domain=['M', 'F'], rangeMin=30, rangeMax=180)
            ).properties(
                title=[f'{gse}; N CpGs = {n_cpg}', f'N healthy = {n_HC}, N non-healthy = {n_nHC}'],
                width=330,
                height=330
            )
    else:
        pca_data = pd.DataFrame({'PC1':Xp[:, 0], 'PC2':Xp[:, 1], 
                            'Condition':meta['Condition'], 
                            'Age':meta['Age'],
                            })
                #plot data
        sub = alt.Chart(pca_data).mark_point(clip=True).encode(
                alt.X('PC1', axis=alt.Axis(grid=False, tickSize=4, labels=False))\
                    .title(f'PC1: {round(expvar[0]* 100, 1) }%'),
                alt.Y('PC2', axis=alt.Axis(grid=False, tickSize=4, labels=False))\
                    .title(f'PC2: {round(expvar[1]* 100, 1) }%'),
                color=alt.Color('Age').scale(scheme="reds"),
                shape=alt.Shape('Condition:N'),
            ).properties(
                title=[f'{gse}; N CpGs = {n_cpg}', f'N healthy = {n_HC}, N non-healthy = {n_nHC}'],
                width=330,
                height=330
            )
    sub.save(fname)
    
#     row.append(sub)
#     if len(row) == 3:
#         finalfig.append(alt.hconcat(*row)\
#                         .resolve_scale(
#                             size='independent', 
#                             shape='independent', 
#                             color='independent'
#                             )
#                         )
#         row = []
# chart = alt.vconcat(*finalfig)
# chart.save('figures/supplement/chart.svg')

100%|██████████| 27/27 [05:11<00:00, 11.55s/it]
